<a href="https://colab.research.google.com/github/Abhishekcq/trial/blob/main/regression_bear_consumption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np
import pandas as pd
import re
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor, HuberRegressor

In [2]:
data = pd.read_csv('/content/Consumo_cerveja.csv')

In [3]:
data

,Data,Temperatura Media (C),Temperatura Minima (C),Temperatura Maxima (C),Precipitacao (mm),Final de Semana,Consumo de cerveja (litros)
0,2015-01-01,"27,3","23,9","32,5",0,0.0,25.461
1,2015-01-02,"27,02","24,5","33,5",0,0.0,28.972
2,2015-01-03,"24,82","22,4","29,9",0,1.0,30.814
3,2015-01-04,"23,98","21,5","28,6","1,2",1.0,29.799
4,2015-01-05,"23,82",21,"28,3",0,0.0,28.900
...,...,...,...,...,...,...,...
936,NaN,NaN,NaN,NaN,NaN,NaN,NaN
937,NaN,NaN,NaN,NaN,NaN,NaN,NaN
938,NaN,NaN,NaN,NaN,NaN,NaN,NaN
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 941 entries, 0 to 940
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Data                         365 non-null    object 
 1   Temperatura Media (C)        365 non-null    object 
 2   Temperatura Minima (C)       365 non-null    object 
 3   Temperatura Maxima (C)       365 non-null    object 
 4   Precipitacao (mm)            365 non-null    object 
 5   Final de Semana              365 non-null    float64
 6   Consumo de cerveja (litros)  365 non-null    float64
dtypes: float64(2), object(5)
memory usage: 51.6+ KB


In [6]:
data.shape

(941, 7)

In [7]:
data.isna().mean()

Data                           0.612115
Temperatura Media (C)          0.612115
Temperatura Minima (C)         0.612115
Temperatura Maxima (C)         0.612115
Precipitacao (mm)              0.612115
Final de Semana                0.612115
Consumo de cerveja (litros)    0.612115
dtype: float64

In [8]:
def preprocess_inputs(df):
    df = df.copy()
    
    # Drop missing rows
    df = df.dropna(axis=0).reset_index(drop=True)
    
    # Replace , with . in numeric columns
    for column in ['Temperatura Media (C)', 'Temperatura Minima (C)' ,'Temperatura Maxima (C)', 'Precipitacao (mm)']:
        df[column] = df[column].apply(lambda x: np.float(re.sub(r',', '.', x)))
    
    # Create date features
    df['Data'] = pd.to_datetime(df['Data'])

    df['Month'] = df['Data'].apply(lambda x: x.month)
    df['Day'] = df['Data'].apply(lambda x: x.day)
    
    df = df.drop('Data', axis=1)
    
    # Split df into X and y
    y = df['Consumo de cerveja (litros)'].copy()
    X = df.drop('Consumo de cerveja (litros)', axis=1).copy()
    
    # Scale X with a standard scaler
    scaler = StandardScaler()
    X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
    
    return X, y

In [11]:
X, y = preprocess_inputs(data)

In [13]:
X

,Temperatura Media (C),Temperatura Minima (C),Temperatura Maxima (C),Precipitacao (mm),Final de Semana,Month,Day
0,1.912508,2.281333,1.365781,-0.419062,-0.631243,-1.602745,-1.673503
1,1.824340,2.493924,1.597722,-0.419062,-0.631243,-1.602745,-1.559818
2,1.131590,1.749853,0.762735,-0.419062,1.584177,-1.602745,-1.446134
3,0.867085,1.430966,0.461212,-0.322294,1.584177,-1.602745,-1.332449
4,0.816703,1.253806,0.391630,-0.419062,-0.631243,-1.602745,-1.218764
...,...,...,...,...,...,...,...
360,0.873383,1.289238,0.368436,0.677640,1.584177,1.587648,1.282303
361,0.445137,1.289238,0.020525,-0.419062,-0.631243,1.587648,1.395988
362,0.142846,1.005782,-0.582521,0.411528,-0.631243,1.587648,1.509672
363,0.048380,0.651463,-0.976820,0.088969,-0.631243,1.587648,1.623357


TRAINING

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=123)

In [16]:
models = {
    '    Linear Regression': LinearRegression(),
    '     Ridge Regression': Ridge(),
    '     Lasso Regression': Lasso(),
    'ElasticNet Regression': ElasticNet(),
    '       SGD Regression': SGDRegressor(),
    '     Huber Regression': HuberRegressor()}

In [17]:
for model in models.values():
    model.fit(X_train, y_train)

In [18]:
for name, model in models.items():
    print(name + " R^2 Score: {:.4f}".format(model.score(X_test, y_test)))

    Linear Regression R^2 Score: 0.6896
     Ridge Regression R^2 Score: 0.6909
     Lasso Regression R^2 Score: 0.5763
ElasticNet Regression R^2 Score: 0.5646
       SGD Regression R^2 Score: 0.6927
     Huber Regression R^2 Score: 0.6699
